# Corruption Example

## WHN

In [15]:
from lib.corruption import WHN
from lib.spdataset import SpeechCommandsV2
from tqdm import tqdm

sc2_set = SpeechCommandsV2(
    root_path='/root/data/', mode='testing', download=True,
    data_tf=WHN(lsnr=6, rsnr=7, step=.5)
)

for feature, label in tqdm(sc2_set):
    pass

100%|██████████| 11005/11005 [00:03<00:00, 3320.66it/s]


## PSH

In [16]:
from lib.corruption import DynPSH
from lib.dataset import GpuMultiTFDataset

# Without the GPU version:
# sc2_set = SpeechCommandsV2(
#     root_path='/root/data', mode='testing', download=True,
#     data_tf=DynPSH(sample_rate=16000, min_steps=4, max_steps=5, is_bidirection=True)
# )

# Using GPU to speed up the PSH proessing:
sc2_set = GpuMultiTFDataset(
    dataset=SpeechCommandsV2(
        root_path='/root/data', mode='testing', download=True
    ), device='cuda', maintain_cpu=True,
    tfs=[
        DynPSH(sample_rate=16000, min_steps=4, max_steps=5, is_bidirection=True)
    ]
)

for feature, label in tqdm(sc2_set):
    pass

100%|██████████| 11005/11005 [02:51<00:00, 64.25it/s]


## TST

In [17]:
from lib.corruption import DynTST
from torch.utils.data import DataLoader
from lib.component import Components, AudioPadding

# One thread version:
# sc2_set = SpeechCommandsV2(
#     root_path='/root/data', mode='testing', download=True,
#     data_tf=DynTST(min_rate=.04, max_rate=.06, step=.01)
# )

# Using multiple CPU cores to speed up
## Note: batch process needs sample audio length, so we padding samples
sc2_set = SpeechCommandsV2(
    root_path='/root/data', mode='testing', download=True,
    data_tf=Components(transforms=[
        DynTST(min_rate=.04, max_rate=.06, step=.01),
        AudioPadding(max_length=16000, sample_rate=16000, random_shift=False)
    ])
)
sc2_loader = DataLoader(
    dataset=sc2_set, batch_size=32, shuffle=False, drop_last=False,
    num_workers=16
)

for features, labels in tqdm(sc2_loader):
    pass

100%|██████████| 344/344 [00:18<00:00, 18.85it/s]


## ENSC

In [18]:
from lib.corruption import DynEN, ensc_noises

noise_list = ensc_noises(
    ensc_path='/root/data', noise_modes=['exercise_bike', 'running_tap', 'white_noise', 'pink_noise'], 
    sample_rate=16000
)
sc2_set = SpeechCommandsV2(
    root_path='/root/data', mode='testing', download=True,
    data_tf=DynEN(noise_list=noise_list, lsnr=5, rsnr=6, step=.5)
)

for feature, label in tqdm(sc2_set):
    pass

Loading noise files...


100%|██████████| 6/6 [00:00<00:00, 129.33it/s]


TTL noise size is: 4


100%|██████████| 11005/11005 [00:02<00:00, 4013.44it/s]


# ENQ

In [19]:
from lib.corruption import enq_noises

# You need download QUT-NOISE dataset by yourself
noise_list = enq_noises(
    enq_path='/root/data/QUT-NOISE', noise_modes=['HOME', 'REVERB', 'STREET'], sample_rate=16000
)
sc2_set = SpeechCommandsV2(
    root_path='/root/data', mode='testing', download=True,
    data_tf=DynEN(noise_list=noise_list, lsnr=5, rsnr=6, step=.5)
)

for feature, label in tqdm(sc2_set):
    pass

Loading noise files...


100%|██████████| 12/12 [00:40<00:00,  3.37s/it]


TTL noise size is: 12


100%|██████████| 11005/11005 [00:02<00:00, 4002.14it/s]


## END1

In [20]:
from lib.corruption import end_noises

# You need download DEMAND dataset by yourself
noise_list = end_noises(
    end_path='/root/data/DEMAND_16k', noise_modes=['NFIELD', 'PRESTO', 'TCAR', 'OOFFICE'],
    sample_rate=16000
)
sc2_set = SpeechCommandsV2(
    root_path='/root/data', mode='testing', download=True,
    data_tf=DynEN(noise_list=noise_list, lsnr=5, rsnr=6, step=.5)
)

for feature, label in tqdm(sc2_set):
    pass

Loading noise files...


100%|██████████| 64/64 [00:00<00:00, 76.40it/s]


TTL noise size is: 64


100%|██████████| 11005/11005 [00:02<00:00, 4051.06it/s]


## END2

In [21]:
# You need download DEMAND dataset by yourself
noise_list = end_noises(
    end_path='/root/data/DEMAND_16k', noise_modes=['DLIVING', 'OHALLWAY', 'SPSQUARE', 'TMETRO'],
    sample_rate=16000
)
sc2_set = SpeechCommandsV2(
    root_path='/root/data', mode='testing', download=True,
    data_tf=DynEN(noise_list=noise_list, lsnr=5, rsnr=6, step=.5)
)

for feature, label in tqdm(sc2_set):
    pass

Loading noise files...


100%|██████████| 64/64 [00:00<00:00, 75.90it/s]


TTL noise size is: 64


100%|██████████| 11005/11005 [00:02<00:00, 4028.86it/s]
